In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [8]:
resultDir = 'results'
trainingSetSize = 'small' # choices=['small', 'large']
problem = 'cauctions' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small', 'medium', 'big']
sampling_Strategies = ['uniform5','depthK'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [9]:
# f1 = "cauctions_20210710-004536.csv"
# f2 = "cauctions_20210710-095340.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
## df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

df = pd.read_csv(f'{resultDir}/{trainingSetSize}/{problem}/{problem}_eval.csv')

In [5]:
# f1 = "setcover_20210725-205924.csv"
# f2 = "setcover_20210726-163252.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
# df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

In [10]:
# eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
# eval_file = eval_files[-1]
# eval_file = f"{resultDir}/cauctions_20210710-004536.csv"

# df = pd.read_csv(eval_file)
# df_list = []
# [df[df['type'] == probSize] for probSize in problemSizes]

# df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [11]:
'a' + 'b'

'ab'

In [12]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

In [13]:
df_gcnns

,policy,sampling_strategy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:baseline,uniform5,0,small,data/instances/cauctions/transfer_100_500/inst...,25.0,83.0,1.36,0.0,optimal,0,0,1.631033,1.625000
1,gcnn:baseline,uniform5,1,small,data/instances/cauctions/transfer_100_500/inst...,25.0,84.0,1.17,0.0,optimal,0,0,1.207962,1.187500
2,gcnn:baseline,uniform5,2,small,data/instances/cauctions/transfer_100_500/inst...,23.0,85.0,0.89,0.0,optimal,0,0,0.948778,0.968750
3,gcnn:baseline,uniform5,3,small,data/instances/cauctions/transfer_100_500/inst...,21.0,74.0,0.97,0.0,optimal,0,0,1.006422,1.000000
4,gcnn:baseline,uniform5,4,small,data/instances/cauctions/transfer_100_500/inst...,21.0,77.0,0.92,0.0,optimal,0,0,0.932163,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,gcnn:baseline,depthK,0,big,data/instances/cauctions/transfer_300_1500/ins...,18983.0,21356.0,241.16,0.0,optimal,0,0,257.070510,251.312500
596,gcnn:baseline,depthK,1,big,data/instances/cauctions/transfer_300_1500/ins...,15232.0,17528.0,193.49,0.0,optimal,0,0,206.886415,202.031250
597,gcnn:baseline,depthK,2,big,data/instances/cauctions/transfer_300_1500/ins...,22482.0,25445.0,276.80,0.0,optimal,0,0,296.552244,288.140625
598,gcnn:baseline,depthK,3,big,data/instances/cauctions/transfer_300_1500/ins...,15442.0,18141.0,194.71,0.0,optimal,0,0,208.420291,203.359375


# 1. Means

In [14]:
dfgcnns_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes       stime
type   sampling_strategy                          
big    depthK             12616.356684  170.875465
       uniform5           10771.800930  147.067744
medium depthK               860.438336   14.555441
       uniform5             866.559059   14.587042
small  depthK                71.143195    1.663572
       uniform5              69.503614    1.655871

In [15]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'uniform5')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [16]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             1.171239  1.161883
       uniform5           1.000000  1.000000
medium depthK             0.992937  0.997834
       uniform5           1.000000  1.000000
small  depthK             1.023590  1.004651
       uniform5           1.000000  1.000000

# 2. Std variances (per instance)

In [17]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [18]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             0.281520  0.264599
       uniform5           0.115072  0.103466
medium depthK             0.152328  0.107858
       uniform5           0.136777  0.091954
small  depthK             0.158182  0.088725
       uniform5           0.112447  0.071717

# 做一下 T-Test

# 计算Wins

In [19]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'uniform5'),metric]
        for sampling_strategy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,sampling_strategy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,sampling_strategy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,sampling_strategy),(metric,)] = [t_stats, p_value]

In [20]:
ttest_res

nnodes               stime          
                           t_stats   p_value   t_stats   p_value
type   sampling_strategy                                        
big    depthK             9.663303       0.0  9.959918       0.0
       uniform5                0.0       1.0       0.0       1.0
medium depthK             2.066653  0.040068  2.331859  0.020713
       uniform5                0.0       1.0       0.0       1.0
small  depthK            -0.602698  0.547399 -0.024666  0.980346
       uniform5                0.0       1.0       0.0       1.0

In [21]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type   instance                                                  seed
big    data/instances/cauctions/transfer_300_1500/instance_1.lp  0       400.0
                                                                 1       401.0
                                                                 2       402.0
                                                                 3       408.0
                                                                 4       409.0
                                                                         ...  
small  data/instances/cauctions/transfer_100_500/instance_9.lp   0        80.0
                                                                 1        86.0
                                                                 2        87.0
                                                                 3        88.0
                                                                 4        89.0
Name: stime, Length: 300, dtype: float64

In [22]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [23]:
len(winner_idx)

300

In [24]:
dfgcnns_wins = df_gcnns.groupby(['type', 'sampling_strategy'])['wins'].sum()
dfgcnns_wins

type    sampling_strategy
big     depthK               35
        uniform5             65
medium  depthK               55
        uniform5             45
small   depthK               46
        uniform5             54
Name: wins, dtype: int64

In [25]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [26]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                            nnodes     stime    nnodes     stime wins
type   sampling_strategy                                             
big    depthK             1.171239  1.161883  0.281520  0.264599   35
       uniform5           1.000000  1.000000  0.115072  0.103466   65
medium depthK             0.992937  0.997834  0.152328  0.107858   55
       uniform5           1.000000  1.000000  0.136777  0.091954   45
small  depthK             1.023590  1.004651  0.158182  0.088725   46
       uniform5           1.000000  1.000000  0.112447  0.071717   54

In [27]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((problemSizes, sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx

MultiIndex([( 'small', 'uniform5'),
            ( 'small',   'depthK'),
            ('medium', 'uniform5'),
            ('medium',   'depthK'),
            (   'big', 'uniform5'),
            (   'big',   'depthK')],
           names=['type', 'sampling_strategy'])

In [28]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [29]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   sampling_strategy                                  
small  uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
medium uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
big    uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN

In [30]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [31]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [32]:
out_df1

nnodes             stime  wins  \
type   sampling_strategy                                             
small  uniform5           1.0000r ± 11.24%   1.0000r ± 7.17%    54   
       depthK             1.0236r ± 15.82%   1.0047r ± 8.87%    46   
medium uniform5           1.0000r ± 13.68%   1.0000r ± 9.20%    45   
       depthK             0.9929r ± 15.23%  0.9978r ± 10.79%    55   
big    uniform5           1.0000r ± 11.51%  1.0000r ± 10.35%    65   
       depthK             1.1712r ± 28.15%  1.1619r ± 26.46%    35   

                         t-stats (p-val)  
type   sampling_strategy                  
small  uniform5             0.00(1.0000)  
       depthK              -0.02(0.9803)  
medium uniform5             0.00(1.0000)  
       depthK               2.33(0.0207)  
big    uniform5             0.00(1.0000)  
       depthK               9.96(0.0000)